# OpenAI Whisper - CPU
Improving CPU-deployment performance of OpenAI Whisper model, following this procedure:
https://pytorch.org/assets/images/quantization-practice/quantization-flowchart2.png

## Load Model

In [66]:
import whisper
import torch

test_path  = "C:\\Users\\win8t\\Music\\"
test_path += "Fugees - Killing Me Softly With His Song (Official Video).mp3"

model_fp32 = whisper.load_model(
    name="small",
    device="cuda")

## Dynamically Quantize Model

In [67]:
print(torch.__version__)

1.10.0+cu113


In [68]:
quantized_model = torch.quantization.quantize_dynamic(
    model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)

NotImplementedError: Could not run 'quantized::linear_prepack' with arguments from the 'QuantizedCUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear_prepack' is only available for these backends: [QuantizedCPU, BackendSelect, Python, Named, Conjugate, Negative, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, Tracer, UNKNOWN_TENSOR_TYPE_ID, Autocast, Batched, VmapMode].

QuantizedCPU: registered at ..\aten\src\ATen\native\quantized\cpu\qlinear_prepack.cpp:324 [kernel]
BackendSelect: fallthrough registered at ..\aten\src\ATen\core\BackendSelectFallbackKernel.cpp:3 [backend fallback]
Python: registered at ..\aten\src\ATen\core\PythonFallbackKernel.cpp:47 [backend fallback]
Name

In [58]:
print(quantized_model)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0): ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (key): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (value): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (out): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): DynamicQuantizedLinear(in_features=512, out_features=2048, dtype=torch.qint8, qsc

In [69]:
print(model_fp32)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0): ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=False)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (out): Linear(in_features=768, out_features=768, bias=True)
        )
        (attn_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (mlp_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): ResidualAttentionBlock(
        (attn): MultiHeadAttention(
  

In [70]:
import os

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")/1e6
    print('Size (MB):', size)
    os.remove('temp.p')
    return size

print_size_of_model(model_fp32)
print_size_of_model(quantized_model)

Size (MB): 967.111363
Size (MB): 158.410839


158.410839

## Run Dynamically Quantized Model

In [71]:
audio = whisper.load_audio(test_path)
audio = whisper.pad_or_trim(audio)

mel   = whisper.log_mel_spectrogram(audio).to(model_fp32.device)
options = whisper.DecodingOptions(fp16=False)

In [72]:
# regular
_, probs = model_fp32.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


In [73]:
# quantized
_, probs = quantized_model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [74]:
import time
def time_model_evaluation(model, mel, options):
    eval_start_time = time.time()
    result = whisper.decode(model, mel, options)
    eval_end_time = time.time()
    eval_duration_time = eval_end_time - eval_start_time
    print(result.text)
    print("Evaluate total time (seconds): {0:.1f}".format(eval_duration_time))

# Evaluate the original FP32 BERT model
time_model_evaluation(model_fp32, mel, options)

# Evaluate the INT8 BERT model after the dynamic quantization
# time_model_evaluation(quantized_model, mel, options)

Strumming my pain with his fingers, singing my life with his words. Killing me softly with his song, killing me softly, with his song telling my whole life,
Evaluate total time (seconds): 0.7
